### Создание бинарного файла

In [1]:
import os
import random
import struct

def create_binary_file(file_name, size_gb):
    size_bytes = size_gb * 1024**3
    count = size_bytes // 4  # Число 32-разрядных чисел в файле

    with open(file_name, 'wb') as f:
        for _ in range(count):
            num = random.getrandbits(32) 
            f.write(struct.pack('>I', num))

create_binary_file('random_numbers.bin', 2)

Создание бинарного файла заняло ~1 минуту 16 секунд.

### Последовательное чтение

In [2]:
def analyze_binary_file(file_name):
    min_num = float('inf')
    max_num = float('-inf')
    total_sum = 0
    
    with open(file_name, 'rb') as f:
        while chunk := f.read(4):
            (num,) = struct.unpack('>I', chunk)
            min_num = min(min_num, num)
            max_num = max(max_num, num)
            total_sum += num
    
    return total_sum, min_num, max_num

# Анализируем созданный файл
total_sum, min_num, max_num = analyze_binary_file('random_numbers.bin')
print(f'Сумма: {total_sum}, Минимум: {min_num}, Максимум: {max_num}')

Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292


Последовательное чтение из файла заняло около **~2 минуты 35 секунд**.
Интересно, что максимальное число в файле отличеатся от абсолютного максимума для 32 разрядного числа всего на **4**.

### Параллельное чтение

In [3]:
import mmap
import concurrent.futures

def process_chunk(start, size, file_name):
    min_num = float('inf')
    max_num = float('-inf')
    total_sum = 0
    
    with open(file_name, 'rb') as f:
        mmapped_file = mmap.mmap(f.fileno(), length=size, offset=start, access=mmap.ACCESS_READ)
        
        for i in range(0, size, 4):
            (num,) = struct.unpack('>I', mmapped_file[i:i+4])
            min_num = min(min_num, num)
            max_num = max(max_num, num)
            total_sum += num
        
        mmapped_file.close()
    
    return total_sum, min_num, max_num

def analyze_binary_file_multithreaded(file_name, num_threads=8):
    file_size = os.path.getsize(file_name)
    chunk_size = file_size // num_threads
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(process_chunk, i * chunk_size,
                            chunk_size if i != num_threads - 1 else file_size - i * chunk_size,
                            file_name)
            for i in range(num_threads)
        ]
        
        total_sum, min_num, max_num = 0, float('inf'), float('-inf')
        for future in concurrent.futures.as_completed(futures):
            ts, mn, mx = future.result()
            total_sum += ts
            min_num = min(min_num, mn)
            max_num = max(max_num, mx)
    
    return total_sum, min_num, max_num


total_sum_mt, min_num_mt, max_num_mt = analyze_binary_file_multithreaded('random_numbers.bin')
print(f'Сумма: {total_sum_mt}, Минимум: {min_num_mt}, Максимум: {max_num_mt}')

Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292


Считывание в 8 поток занло 2 минуты 12 секунд.

In [4]:
import time

for threads_num in range(8, 33, 4):
    start_time = time.time()
    total_sum_mt, min_num_mt, max_num_mt = analyze_binary_file_multithreaded('random_numbers.bin')
    end_time = time.time()
    print(f'\tСумма: {total_sum_mt}, Минимум: {min_num_mt}, Максимум: {max_num_mt}')
    print(f'Многопоточное чтение с количеством потоков, равным {threads_num}, заняло {end_time - start_time:.2f} секунд')

	Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение с количеством потоков, равным 8, заняло 131.78 секунд
	Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение с количеством потоков, равным 12, заняло 127.16 секунд
	Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение с количеством потоков, равным 16, заняло 126.25 секунд
	Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение с количеством потоков, равным 20, заняло 127.00 секунд
	Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение с количеством потоков, равным 24, заняло 127.21 секунд


На питоне разница не сильно заметна. Помимо питона я решил попробовать использовать с++. Файл multiprocess_reading.cpp запускается командами:

```
g++ multiprocess_reading.cpp -o main -std=c++11 -stdlib=libc++ && ./main 
```

Программа выводит ответ 

```
Последовательное чтение заняло 10 секунд
Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
Многопоточное чтение заняло 1 секунд
Сумма: 1152929983093036970, Минимум: 5, Максимум: 4294967292
```
Некоторые выводы:

* Тоесть последовательное чтение на с++ в 13 раз быстрее чем последовательное чтение на python.
* Многопоточное чтение на c++ в 10 раз быстрее чем последовательное на c++.
* И наконец, многопоточное чтение на c++ в 130 раз быстрее чам последоавтальное на python :)